In [1]:
# Imports
import pandas as pd
import os

In [2]:
# Helper functions
def get_table_name(file_name):
    first_occurrence = file_name.find("_")

    if first_occurrence != -1:
        second_occurrence = file_name.find("_", first_occurrence + 1)
        if second_occurrence != -1:
            return file_name[0:second_occurrence]
        else:
            return file_name.replace(".csv", "")
    else:
        return file_name.replace(".csv", "")

def merge_columns(df, name, series_to_merge):
    output = df[series_to_merge[0]]
    df.drop(columns=series_to_merge[0], inplace=True)
    series_to_merge[0]
    for series in series_to_merge[1:]:
        try: 
            output.update(df[series])
            df.drop(columns=series, inplace=True)
        except:
            print(series)
    df[name] = output
    return df
        
        

In [3]:
# Importing tables
tables = {}
table_root = "./tables/"
count = 0
# The first columns of tables that should NOT be joined
exempt_first_columns = ["Temperature °C", "Temperature (°C)", "Altitude (m)"]
for file in os.listdir(table_root):
    file_table = pd.read_csv(table_root + file, sep=',', index_col=False)
    # Removing the empty final column that came from the trailing comma in each file
    final_col_index = len(file_table.columns) - 1
    # 
    if (not (file_table.columns[0] in exempt_first_columns)):
        tables[file] = file_table.iloc[:, 0:final_col_index]

In [4]:
# Exploring ways to join tables
columns = {}
for table_key in tables.keys():
    table = tables.get(table_key)
    first_column = table.columns[0]
    if not (first_column in columns):
        columns.update({first_column: 1})
    else: 
        count = columns.get(first_column)
        count = count + 1
        columns.update({first_column: count})
for column in columns.keys():
    print(column + " - " + str(columns.get(column)))

Aluminum alloy - 14
Name - 121
Metal - 12
Gas - 9
Wood - 8
Liquid - 13
Trade name - 33
Rock - 1
Material - 23
Alloy - 26
Soil type - 3
Mineral - 2
Rocks - 2
Semiconductor - 7
Halide - 2
Platinum alloy - 1
Alcohol - 1
Fuel - 3
Ester - 2
Ether - 2


In [5]:
# Joining all the applicable tables
start = False
big_table = pd.DataFrame()
for table_key in tables.keys():
    if table.columns[0] not in exempt_first_columns:
        try:
            if start:
                table = tables.get(table_key)
                table_name = get_table_name(table_key)
                table = table.rename(columns={table.columns[0]: "Name"})
                table = table.astype({"Name": "string"})
                big_table = pd.merge(
                    left = big_table,
                    right = table,
                    on = "Name",
                    suffixes = ["", "_" + table_key ],
                    how = 'outer'
                )
            else:
                start = True
                big_table = tables.get(table_key)
                big_table = big_table.astype({big_table.columns[0]: "string"})
                big_table = big_table.rename(columns={big_table.columns[0]: "Name"})
        except Exception as ex:
            print("ERROR: " + "table_key: " + table_key + ". Error Message: " + str(ex))
big_table.head()

,Name,Thermal expansion,Temperature,Refractive index (n,Specific heat capacity,Thermal conductivity,Dynamic viscosity,Thermal expansion_Thermal_Expansion_of_Pure_Metals.csv,Tensile strength [perpendicular to grain],UNS number,...,Thermal expansion_Linear_Thermal_Expansion_of_Ceramics.csv,UNS number_Electrical_Conductivity_of_Nickel_Alloys.csv,(%IACS)_Electrical_Conductivity_of_Nickel_Alloys.csv,(S/m) ×10_Electrical_Conductivity_of_Nickel_Alloys.csv,Density_Density_of_Precious_Metals.csv,UNS number_Melting_Point_of_Magnesium_Alloys.csv,Melting point_Melting_Point_of_Magnesium_Alloys.csv,Specific heat capacity_Specific_Heat_Capacity_of_Soils.csv,UNS number_Specific_Heat_Capacity_of_Nickel_Alloys.csv,Specific heat capacity_Specific_Heat_Capacity_of_Nickel_Alloys.csv
0,1050,23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1060,23.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1100,23.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1199,23.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011,22.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
print(big_table.shape)
columns = big_table.columns
columns = list(columns)
columns.sort()
for column in columns:
    print(column)

(61912, 439)
(%IACS)
(%IACS)_Electrical_Conductivity_of_Cast_Copper_Alloys.csv
(%IACS)_Electrical_Conductivity_of_Magnesium_Alloys.csv
(%IACS)_Electrical_Conductivity_of_Nickel_Alloys.csv
(%IACS)_Electrical_Conductivity_of_Titanium_Alloys.csv
(%IACS)_Electrical_Conductivity_of_Wrought_Aluminum_Alloys.csv
(%IACS)_Electrical_Conductivity_of_Wrought_Copper_Alloys.csv
(%IACS)_Electrical_Conductivity_of_Zinc_Alloys.csv
(BTU/lb)
(MJ/kg)
(S/m) ×10
(S/m) ×10_Electrical_Conductivity_of_Cast_Copper_Alloys.csv
(S/m) ×10_Electrical_Conductivity_of_Magnesium_Alloys.csv
(S/m) ×10_Electrical_Conductivity_of_Nickel_Alloys.csv
(S/m) ×10_Electrical_Conductivity_of_Titanium_Alloys.csv
(S/m) ×10_Electrical_Conductivity_of_Wrought_Aluminum_Alloys.csv
(S/m) ×10_Electrical_Conductivity_of_Wrought_Copper_Alloys.csv
(S/m) ×10_Electrical_Conductivity_of_Zinc_Alloys.csv
API gravity
Additional_Info
Boiling point
Boiling point_Boiling_Point_of_Acids.csv
Boiling point_Boiling_Point_of_Alcohols.csv
Boiling point_Boi

In [7]:
# Find one off tables
# for table_key in tables.keys():
#    table = tables.get(table_key)
#    if 'Density' in table.columns:
#        print(table_key)

In [17]:
# Investigate a column
big_table[big_table['Resistivity (Ω.m)'].notnull()][["Name", 'Resistivity (Ω.m)']]

,Name,Resistivity (Ω.m)
156,Air,109 - 1016
181,Hydrogen,106 - 109
59494,Aluminum,2.65 ×10-8 - 2.82 ×10-8
59499,Copper,1.7 ×10-8
59500,Gold,2.44 ×10-8
59501,Iron,9.7 ×10-8
59502,Lead,2.2 ×10-7
59504,Mercury,9.8 ×10-7
59505,Nickel,6.99 ×10-8
59506,Platinum,1.06 ×10-7


In [9]:
# Holding
'Resistivity', - Titanium
'Resistivity (Ω.m)', - Common materials
'Resistivity (Ω·m) ×10', - Pure metals
'Resistivity_Electrical_Resistivity_of_Ceramics.csv', 
'Resistivity_Electrical_Resistivity_of_Dielectric_Materials.csv', 
'Resistivity_Electrical_Resistivity_of_Magnesium_Alloys.csv', 
'Resistivity_Electrical_Resistivity_of_Nickel_Alloys.csv', 
'Resistivity_Electrical_Resistivity_of_Platinum_Alloys.csv', 
'Resistivity_Electrical_Resistivity_of_Polymers_and_Plastics.csv', 
'Resistivity_Electrical_Resistivity_of_Precious_Metals.csv', 
'Resistivity_Electrical_Resistivity_of_Silver_Alloys.csv', 
'Resistivity_Electrical_Resistivity_of_Zinc_Alloys.csv'

In [10]:
# Merging columns
big_table = merge_columns(big_table, "Electrical_Conductivity_(%IACS)", ['(%IACS)', 'Conductivity (%IACS)', '(%IACS)_Electrical_Conductivity_of_Cast_Copper_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Magnesium_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Nickel_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Titanium_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Wrought_Aluminum_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Wrought_Copper_Alloys.csv', '(%IACS)_Electrical_Conductivity_of_Zinc_Alloys.csv'])
big_table = merge_columns(big_table, "Electrical_Conductivity_(S/m)", ['Conductivity (S/m)', 'Conductivity (S/m)_Electrical_Conductivity_Values_for_Common_Materials.csv',])
big_table = merge_columns(big_table, "Electrical_Conductivity_(S/cm)", ['Conductivity (S/cm)'])
big_table = merge_columns(big_table, "Electrical_Conductivity_(S/m)*10^6", ['Conductivity (S/m) ×10', '(S/m) ×10_Electrical_Conductivity_of_Magnesium_Alloys.csv', '(S/m) ×10_Electrical_Conductivity_of_Nickel_Alloys.csv', 'Conductivity (S/m) ×10_Electrical_Conductivity_of_Silver_Alloys.csv', '(S/m) ×10_Electrical_Conductivity_of_Titanium_Alloys.csv', '(S/m) ×10_Electrical_Conductivity_of_Zinc_Alloys.csv', 'Conductivity (S/m) ×10_Electrical_Conductivity_of_Precious_Metals.csv', ])
big_table = merge_columns(big_table, "Electrical_Conductivity_(S/m)*10^7", ['(S/m) ×10_Electrical_Conductivity_of_Wrought_Aluminum_Alloys.csv', '(S/m) ×10', '(S/m) ×10_Electrical_Conductivity_of_Cast_Copper_Alloys.csv', '(S/m) ×10_Electrical_Conductivity_of_Wrought_Copper_Alloys.csv',])
big_table = merge_columns(big_table, "Boiling_point(C)", ['Boiling point', 'Boiling point_Boiling_Point_of_Acids.csv', 'Boiling point_Boiling_Point_of_Alcohols.csv', 'Boiling point_Boiling_Point_of_Alkanes.csv', 'Boiling point_Boiling_Point_of_Alkenes.csv', 'Boiling point_Boiling_Point_of_Alkynes.csv', 'Boiling point_Boiling_Point_of_Bases.csv', 'Boiling point_Boiling_Point_of_Common_Substances.csv', 'Boiling point_Boiling_Point_of_Esters.csv', 'Boiling point_Boiling_Point_of_Ethers.csv', 'Boiling point_Boiling_Point_of_Gases.csv', 'Boiling point_Boiling_Point_of_Halides.csv', 'Boiling point_Boiling_Point_of_Liquids.csv', 'Boiling point_Boiling_Point_of_Oxides.csv', 'Boiling point_Boiling_Point_of_Precious_Metals.csv', 'Boiling point_Boiling_Point_of_Salts.csv'])
big_table = merge_columns(big_table, "Brinell Hardness (HB)", ['Brinell Hardness (HB)', 'Brinell hardness (HB)', 'Brinell hardness (HB)_Hardness_of_Cast_Copper_Alloys.csv', 'Brinell hardness (HB)_Hardness_of_Magnesium_Alloys_(Annealed).csv', 'Brinell hardness (HB)_Hardness_of_Wrought_Aluminum_Alloys.csv', 'Brinell hardness (HB)_Hardness_of_Wrought_Copper_Alloys.csv', 'Brinell hardness (HB)_Hardness_of_Wrought_Titanium_Alloys_(Annealed).csv', 'Brinell hardness (HB)_Hardness_of_Zinc_Alloys_(Annealed).csv'])
big_table = merge_columns(big_table, "Class_of_Ceramic", ['Class', 'Class_Density_of_Ceramics.csv', 'Class_Electrical_Conductivity_of_Ceramics.csv', 'Class_Electrical_Conductivity_of_Dielectric_Materials.csv', 'Class_Electrical_Resistivity_of_Ceramics.csv', 'Class_Electrical_Resistivity_of_Dielectric_Materials.csv', 'Class_Linear_Thermal_Expansion_of_Ceramics.csv', 'Class_Loss_Tangent_of_Dielectric_Materials.csv', 'Class_Melting_Point_of_Ceramics.csv', 'Class_Relative_Permittivity_of_Dielectric_Materials_at_(1_MHz).csv', 'Class_Thermal_Conductivity_of_Ceramics.csv'])
big_table = merge_columns(big_table, "Composition", ['Composition', 'Composition_Density_of_Titanium_Alloys.csv', 'Composition_Density_of_Zinc_Alloys.csv', 'Composition_Electrical_Conductivity_of_Zinc_Alloys.csv', 'Composition_Electrical_Resistivity_of_Zinc_Alloys.csv', 'Composition_Elongation_of_Magnesium_Alloys_(Annealed).csv', 'Composition_Elongation_of_Titanium_Alloys_(Annealed).csv', 'Composition_Elongation_of_Zinc_Alloys_(Annealed).csv', 'Composition_Hardness_of_Magnesium_Alloys_(Annealed).csv', 'Composition_Hardness_of_Wrought_Titanium_Alloys_(Annealed).csv', 'Composition_Hardness_of_Zinc_Alloys_(Annealed).csv', 'Composition_Linear_Thermal_Expansion_of_Magnesium_Alloys.csv', 'Composition_Linear_Thermal_Expansion_of_Titanium_Alloys.csv', 'Composition_Melting_Point_of_Titanium_Alloys.csv', 'Composition_Melting_Point_of_Zinc_Alloys.csv', 'Composition_Poisson’s_Ratio_of_Titanium_Alloys.csv', 'Composition_Shear_Modulus_of_Titanium_Alloys_(Annealed).csv', 'Composition_Specific_Heat_Capacity_of_Magnesium_Alloys.csv', 'Composition_Specific_Heat_Capacity_of_Titanium_Alloys.csv', 'Composition_Specific_Heat_Capacity_of_Zinc_Alloys.csv', 'Composition_Tensile_Strength_of_Zinc_Alloys.csv', 'Composition_Thermal_Conductivity_of_Magnesium_Alloys.csv', 'Composition_Thermal_Conductivity_of_Titanium_Alloys.csv', 'Composition_Thermal_Conductivity_of_Zinc_Alloys.csv', 'Composition_Young’s_Modulus_Values_for_Titanium_Alloys_(Annealed).csv'])
big_table = merge_columns(big_table, "Compressive_Strength_(MPa)", ['Compressive strength', 'Compressive strength_Compressive_Strength_of_Construction_Materials.csv'])
big_table = merge_columns(big_table, "Thermal_Conductivity_(W/m*K)", ['Conductivity', 'Conductivity_Thermal_Conductivity_of_Cast_Aluminum_Alloys.csv', 'Conductivity_Thermal_Conductivity_of_Ceramics.csv', 'Conductivity_Thermal_Conductivity_of_Magnesium_Alloys.csv', 'Conductivity_Thermal_Conductivity_of_Semiconductors.csv', 'Conductivity_Thermal_Conductivity_of_Titanium_Alloys.csv', 'Conductivity_Thermal_Conductivity_of_Wrought_Aluminum_Alloys.csv'])
big_table = merge_columns(big_table, "Crystal structure", ['Crystal structure', 'Crystal structure_Crystal_Structure_of_Pure_Metals.csv', 'Crystal structure_Density_of_Semiconductors.csv', 'Crystal structure_Linear_Thermal_Expansion_of_Semiconductors.csv', 'Crystal structure_Melting_Point_of_Semiconductors.csv', 'Crystal structure_Refractive_Index_of_Semiconductors.csv', 'Crystal structure_Specific_Heat_Capacity_of_Semiconductors.csv', 'Crystal structure_Thermal_Conductivity_of_Semiconductors.csv'])
big_table = merge_columns(big_table, "Density (kg/m3)", ['Density', 'Density (kg/m3)', 'Density (kg/m3).1', 'Density (kg/m3)_Density_Values_for_Iron_Alloys.csv', 'Density_Densities_of_Common_Materials.csv',  'Density_Density_of_Alkanes.csv', 'Density_Density_of_Alkenes.csv', 'Density_Density_of_Alkynes.csv', 'Density_Density_of_Cast_Aluminum_Alloys.csv', 'Density_Density_of_Cast_Copper_Alloys.csv', 'Density_Density_of_Ceramics.csv', 'Density_Density_of_Common_Minerals.csv', 'Density_Density_of_Common_Rocks.csv', 'Density_Density_of_Common_Wood.csv', 'Density_Density_of_Construction_Materials.csv', 'Density_Density_of_Dielectric_Materials.csv', 'Density_Density_of_Fuels.csv', 'Density_Density_of_Gases.csv', 'Density_Density_of_Halides.csv', 'Density_Density_of_Magnesium_Alloys.csv', 'Density_Density_of_Nickel_Alloys.csv', 'Density_Density_of_Oxides.csv', 'Density_Density_of_Platinum_Alloys.csv', 'Density_Density_of_Polymers_and_Plastics.csv', 'Density_Density_of_Precious_Metals.csv', 'Density_Density_of_Pure_Metals.csv', 'Density_Density_of_Refractory_Materials.csv', 'Density_Density_of_Semiconductors.csv', 'Density_Density_of_Silver_Alloys.csv', 'Density_Density_of_Soils.csv', 'Density_Density_of_Titanium_Alloys.csv', 'Density_Density_of_Wrought_Copper_Alloys.csv', 'Density_Density_of_Zinc_Alloys.csv'])
big_table = merge_columns(big_table, "Density (kg/cm3)", [ 'Density_Density_of_Acids.csv',  'Density_Density_of_Alcohols.csv', 'Density_Density_of_Amino_Acids.csv', 'Density_Density_of_Bases.csv', 'Density_Density_of_Carbohydrates.csv', 'Density_Density_of_Esters.csv', 'Density_Density_of_Ethers.csv', 'Density_Density_of_Liquids.csv', 'Density_Density_of_Phosphates.csv', 'Density_Density_of_Salts.csv', 'Density_Density_of_Sulfides.csv'])
big_table = merge_columns(big_table, "Viscosity_(mPa*s)", ['Dynamic viscosity_Dynamic_Viscosity_of_Liquids.csv', 'Dynamic viscosity_Viscosity_of_Alcohols.csv', 'Dynamic viscosity_Viscosity_of_Alkenes.csv', 'Dynamic viscosity_Viscosity_of_Esters.csv', 'Dynamic viscosity_Viscosity_of_Ethers.csv'])
big_table = merge_columns(big_table, "Viscosity_(microPa*s)", ['Dynamic viscosity', 'Dynamic viscosity_Viscosity_of_Alkanes.csv', ])
big_table = merge_columns(big_table, "Elongation (%)", ['Elongation (%)', 'Elongation (%)_Elongation_of_Magnesium_Alloys_(Annealed).csv', 'Elongation (%)_Elongation_of_Nickel_Alloys_(Annealed).csv', 'Elongation (%)_Elongation_of_Platinum_Alloys_(Annealed).csv', 'Elongation (%)_Elongation_of_Titanium_Alloys_(Annealed).csv'])
big_table = merge_columns(big_table, "Flash_Point_(C)", ['Flash point', 'Flash point (closed cup)', 'Flash point (closed cup)_Flash_Point_of_Fuels.csv', 'Flash point_Flash_Point_of_Esters.csv', 'Flash point_Flash_Point_of_Gases.csv', 'Flash point_Flash_Point_of_Liquids.csv'])
big_table = merge_columns(big_table, "Formula", ['Formula', 'Formula_Molar_Mass_of_Amino_Acids.csv', 'Formula_Molar_Mass_of_Carbohydrates.csv', 'Formula_Molar_Mass_of_Phosphates.csv'])
big_table = merge_columns(big_table, "Melting_Point_(C)", ['Melting point', 'Melting point_Melting_Point_of_Acids.csv', 'Melting point_Melting_Point_of_Alcohols.csv', 'Melting point_Melting_Point_of_Alkanes.csv', 'Melting point_Melting_Point_of_Alkenes.csv', 'Melting point_Melting_Point_of_Alkynes.csv', 'Melting point_Melting_Point_of_Amino_Acids.csv', 'Melting point_Melting_Point_of_Bases.csv', 'Melting point_Melting_Point_of_Carbohydrates.csv', 'Melting point_Melting_Point_of_Cast_Aluminum_Alloys.csv', 'Melting point_Melting_Point_of_Cast_Copper_Alloys.csv', 'Melting point_Melting_Point_of_Ceramics.csv', 'Melting point_Melting_Point_of_Esters.csv', 'Melting point_Melting_Point_of_Ethers.csv', 'Melting point_Melting_Point_of_Halides.csv', 'Melting point_Melting_Point_of_Magnesium_Alloys.csv', 'Melting point_Melting_Point_of_Minerals.csv', 'Melting point_Melting_Point_of_Nickel_Alloys.csv', 'Melting point_Melting_Point_of_Oxides.csv', 'Melting point_Melting_Point_of_Phosphates.csv', 'Melting point_Melting_Point_of_Platinum_Alloys.csv', 'Melting point_Melting_Point_of_Polymers_and_Plastics.csv', 'Melting point_Melting_Point_of_Precious_Metals.csv', 'Melting point_Melting_Point_of_Pure_Metals.csv', 'Melting point_Melting_Point_of_Refractory_Materials.csv', 'Melting point_Melting_Point_of_Salts.csv', 'Melting point_Melting_Point_of_Semiconductors.csv', 'Melting point_Melting_Point_of_Sulfides.csv', 'Melting point_Melting_Point_of_Titanium_Alloys.csv', 'Melting point_Melting_Point_of_Wrought_Aluminum_Alloys.csv', 'Melting point_Melting_Point_of_Wrought_Copper_Alloys.csv', 'Melting point_Melting_Point_of_Zinc_Alloys.csv'])
big_table = merge_columns(big_table, "Modulus_of_Rupture_(MPa)", ['Modulus of rupture', 'Modulus of rupture_Modulus_of_Rupture_of_Common_Rocks.csv'])
big_table = merge_columns(big_table, "Molar_Mass_(g/mol)", ['Molar mass (g/mol)', 'Molar mass (g/mol)_Molar_Mass_of_Acids.csv', 'Molar mass (g/mol)_Molar_Mass_of_Alcohols.csv', 'Molar mass (g/mol)_Molar_Mass_of_Alkanes.csv', 'Molar mass (g/mol)_Molar_Mass_of_Alkenes.csv', 'Molar mass (g/mol)_Molar_Mass_of_Alkynes.csv', 'Molar mass (g/mol)_Molar_Mass_of_Amino_Acids.csv', 'Molar mass (g/mol)_Molar_Mass_of_Bases.csv', 'Molar mass (g/mol)_Molar_Mass_of_Carbohydrates.csv', 'Molar mass (g/mol)_Molar_Mass_of_Esters.csv', 'Molar mass (g/mol)_Molar_Mass_of_Ethers.csv', 'Molar mass (g/mol)_Molar_Mass_of_Gases.csv', 'Molar mass (g/mol)_Molar_Mass_of_Halides.csv', 'Molar mass (g/mol)_Molar_Mass_of_Oxides.csv', 'Molar mass (g/mol)_Molar_Mass_of_Phosphates.csv', 'Molar mass (g/mol)_Molar_Mass_of_Pure_Metals.csv', 'Molar mass (g/mol)_Molar_Mass_of_Salts.csv', 'Molar mass (g/mol)_Molar_Mass_of_Semiconductors.csv', 'Molar mass (g/mol)_Molar_Mass_of_Sulfides.csv'])                       
big_table = merge_columns(big_table, "Poissons_Ratio", ['Poisson’s ratio', 'Poisson’s ratio_Poisson’s_Ratio_of_Titanium_Alloys.csv'])
big_table = merge_columns(big_table, "Refractive_Index_(n_D)", ['Refractive index', 'Refractive index (n', 'Refractive index (n_Refractive_Index_Values_for_Various_Materials.csv', 'Refractive index (n_Refractive_Index_of_Alcohols.csv', 'Refractive index (n_Refractive_Index_of_Alkanes.csv', 'Refractive index (n_Refractive_Index_of_Alkenes.csv', 'Refractive index (n_Refractive_Index_of_Alkynes.csv', 'Refractive index (n_Refractive_Index_of_Amino_Acids.csv', 'Refractive index (n_Refractive_Index_of_Carbohydrates.csv', 'Refractive index (n_Refractive_Index_of_Ethers.csv', 'Refractive index (n_Refractive_Index_of_Polymers_and_Plastics.csv', 'Refractive index_Refractive_Index_of_Gases.csv', 'Refractive index_Refractive_Index_of_Semiconductors.csv'])
big_table = merge_columns(big_table, "Magnetic_Permittivity_micro(H/m)", ['Permeability - μ (H/m)'])
big_table = merge_columns(big_table, "Relative_Permittivity_Dielectric_Constant_(εr)", ['Relative permittivity (εr)', 'Relative permittivity (εr)_Relative_Permittivity_of_Common_Materials.csv', 'Relative permittivity (εr)_Relative_Permittivity_of_Dielectric_Materials_at_(1_MHz).csv'])
big_table = merge_columns(big_table, "", [])
# big_table = merge_columns(big_table, "Thermal_Conductivity_(W/m*K)", [])

/tmp/ipykernel_18672/4101903606.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['5.858' '5.858' '5.336' '5.8' '2.61' '1.276' '1.276' 'SIval' '2.61' '5.8'
 '5.51' '5.394' '1.45' '1.334' '1.624' '1.508' '3.248' '1.624' '1.566'
 '1.856' '1.624' '1.508' '1.856' '1.566' '1.16' '0.87' '0.754' '0.812'
 '0.986' '0.406' '0.812' '0.522' '0.267' '0.522' '0.348' '0.522' '0.348'
 '0.522' 'SIval' '5.568' 'SIval' '0.638' '0.406' '0.696']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  output.update(df[series])
/tmp/ipykernel_18672/4101903606.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Sublimes at 180' '117.9' '51' 'Decomposes at 140' '141' '249' '300'
 '-100' '163.5' '205' 'Decomposes at 250' 'Decomposes at 175' '101' '100'
 '-66.7' '-85' '19.5' '25.6' '150.2' '-41.3' '-60' '-2' 'Decomposes at 0'
 '

Density (kg/m3).1
Density (kg/m3)_Density_Values_for_Iron_Alloys.csv


/tmp/ipykernel_18672/4101903606.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.58' '1.15' '2.5' '2.97' '4.31' '0.592 (at 25°C)' '6.81' '7.36' '14.3'
 '18.8' '53 (at 75°C)' '11.7' '17.2']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  output.update(df[series])
/tmp/ipykernel_18672/4101903606.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['14 - 55' '14 - 55' '14 - 55' '8.3 - 21.4' '10 - 20' '7.4 - 12'
 '9.8 - 19.6' '8 - 31' '12' '5' '2.5' '2 - 50' '34 - 110']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  output.update(df[series])


In [11]:
print(big_table.shape)
columns = big_table.columns
columns = list(columns)
columns.sort()
print(columns)

(61912, 244)
['(BTU/lb)', '(MJ/kg)', 'API gravity', 'Additional_Info', 'Boiling_point(C)', 'Brinell Hardness (HB)', 'Bulk modulus', 'Carbon released (g/MJ)', 'Chemical formula', 'Class_of_Ceramic', 'Composition', 'Compressive_Strength_(MPa)', 'Conductivity (S/m)_Electrical_Conductivity_of_Ceramics.csv', 'Crystal structure', 'Density (kg/cm3)', 'Density (kg/m3)', 'Electrical_Conductivity_(%IACS)', 'Electrical_Conductivity_(S/cm)', 'Electrical_Conductivity_(S/m)', 'Electrical_Conductivity_(S/m)*10^6', 'Electrical_Conductivity_(S/m)*10^7', 'Elongation (%)', 'Expansion', 'Flash_Point_(C)', 'Formula', 'Freezing point', 'Glass transition temperature', 'Loss tangent', 'Melting_Point_(C)', 'Modulus_of_Rupture_(MPa)', 'Mohs hardness', 'Molar_Mass_(g/mol)', 'Name', 'Parallel to grain', 'Permeability - μ (H/m)', 'Perpendicular to grain', 'Poisson’s ratio', 'Poisson’s ratio_Poisson’s_Ratio_of_Titanium_Alloys.csv', 'Porosity (%)', 'Refractive index', 'Refractive index (n', 'Refractive index (n_Refr